In [4]:
import os
import pandas as pd
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

In [10]:
# Rename images
def rename_images(folder):
    counter = 1
    for file_name in os.listdir(folder):
        # Ignore hidden files
        if file_name.startswith('.'):
          continue
    
        # Generate a new file name
        new_file_name = f"img{counter}.jpg"
    
        # Rename the file
        os.rename(os.path.join(folder, file_name), os.path.join(folder, new_file_name))
    
        # Incrementting counter
        counter += 1


In [15]:
#rename_images('../img/Potato/healthy')
#rename_images('../img/Potato/Early_blight')
#rename_images('../img/Potato/Late_blight')

In [52]:
def carregar_dataset_batata(pasta_dataset):
  """
  Função para carregar imagens e classes do dataset de batata.

  Argumentos:
    pasta_dataset: Caminho para a pasta "Potato" que contém as imagens.
  
  Retorna:
    Um DataFrame contendo as imagens e suas classes.
  """

  # Criar um DataFrame vazio
  df = pd.DataFrame(columns=['imagem', 'classe'])

  # Iterar pelas classes (healthy, doença1, doença2)
  for classe in os.listdir(pasta_dataset):
    caminho_classe = os.path.join(pasta_dataset, classe)

    # Iterar pelas imagens na pasta da classe
    for nome_imagem in os.listdir(caminho_classe):
      # Carregar e pré-processar a imagem
      imagem = imread(os.path.join(caminho_classe, nome_imagem))
      imagem = resize(imagem, (224, 224))
      imagem_flatten = imagem.flatten()

      # Adicionar imagem e classe ao DataFrame (using concat)
      nova_linha = pd.DataFrame({'imagem': [imagem_flatten], 'classe': [classe]})
      df = pd.concat([df, nova_linha], ignore_index=True)

  return df

# Carregar o dataset em um DataFrame
df_completo = carregar_dataset_batata('../img/Potato')

# Visualizar o DataFrame
display(df_completo)


,imagem,classe
0,"[0.49481792717086837, 0.48305322128851536, 0.5...",Early_blight
1,"[0.4891956782713085, 0.4813525410164065, 0.536...",Early_blight
2,"[0.6106842737094837, 0.6028411364545818, 0.653...",Early_blight
3,"[0.49045618247298917, 0.47869147659063627, 0.5...",Early_blight
4,"[0.6627050820328131, 0.6587835134053622, 0.713...",Early_blight
...,...,...
2147,"[0.6321928771508604, 0.6165066026410565, 0.671...",Late_blight
2148,"[0.5523209283713486, 0.4307523009203681, 0.422...",Late_blight
2149,"[0.6478991596638655, 0.6204481792717087, 0.647...",Late_blight
2150,"[0.5325130052020808, 0.485454181672669, 0.5011...",Late_blight


In [53]:
df_completo['classe'].replace('healthy', 0, inplace=True)
df_completo['classe'].replace('Early_blight', 1, inplace=True)
df_completo['classe'].replace('Late_blight', 2, inplace=True)


In [54]:
# Divida o DataFrame em conjuntos de treino e teste
X = df_completo[['imagem']]
y = df_completo['classe']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [55]:
# Crie um modelo XGBoost
model = XGBClassifier()

# Treine o modelo com os dados de treino
model.fit(X_train, y_train)


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:imagem: object

In [ ]:
# Normalize os dados usando o StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
